In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv("./train.csv")
features = train.drop(columns="quality")
print(features.keys())
keys = ["fixed acidity", "volatile acidity", "free sulfur dioxide", "density", "pH"]

Index(['type', 'fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'],
      dtype='object')


In [3]:
dim = len(keys)

In [4]:
X_label = np.ndarray(shape=(1 + dim,len(features)),dtype=float)

for i in range(len(features)):
    X_label[0][i] = 1.0
for d, key in enumerate(keys):
    X_label[d+1] = features[key].values

Y_label = train["quality"].values
Y_label = Y_label.reshape((1,len(Y_label)))
# print(X_label.shape,X_label)
# print(Y_label.shape,Y_label)

In [5]:
B = np.where(np.isnan(X_label), 0, X_label).dot(np.where(np.isnan(Y_label.T), 0, Y_label.T))
A = np.where(np.isnan(X_label), 0, X_label).dot(np.where(np.isnan(X_label.T), 0, X_label.T))

L = 1.0 # 正則化の程度を決めるハイパーパラメータ
LI = np.identity(1 + dim) * L

AT = np.linalg.inv(A + LI)
w = np.where(np.isnan(AT), 0, AT).dot(np.where(np.isnan(B), 0, B))
# print(B)
# print(A)
# print(AT)
print(w)

[[ 4.44288378e+00]
 [-4.38985141e-03]
 [-1.54400287e+00]
 [-2.05395608e-03]
 [ 9.04097927e-01]
 [ 3.39102503e-01]]


In [6]:
# output prediction for the test data
test = pd.read_csv("./test.csv")

In [7]:
submission = pd.read_csv("./sample_submission.csv", index_col="id")
submission["quality"] = [w[0][0] + sum(w[i + 1][0] * test[key][n] for i, key in enumerate(keys)) for n in range(len(test))]
submission.to_csv("submission.csv")